In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score

from sklearn.model_selection import KFold

features = [     
    'Make',
    'Model',
    'Year',
    'Engine HP',
    'Engine Cylinders',
    'Transmission Type',
    'Vehicle Style',
    'highway MPG',
    'city mpg',
    'MSRP'
]

df = pd.read_csv('chapter04-eval-data.csv')
df = df[ features ]
df.columns = df.columns.str.replace(' ', '_').str.lower()
df = df.fillna(0)
#df.isnull().sum()

In [2]:
priceMean = df['msrp'].mean()
print( "priceMean = ", priceMean)
df[ 'above_average' ] = (df['msrp'] > priceMean ) + 0
del df['msrp']

priceMean =  40594.737032063116


In [3]:
seed = 1
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=seed)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=seed)

y_train = df_train[ 'above_average' ]
y_val = df_val[ 'above_average' ]

del df_train[ 'above_average' ]
del df_val[ 'above_average' ]


In [4]:
# train and predict functions

def train(df_train, y_train, C=1.0):
    dicts = df_train[categoricalFeatures + numericalFeatures].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

def predict(df, dv, model):
    dicts = df[categoricalFeatures + numericalFeatures].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [5]:
numericalFeatures = [ 'year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg' ]
categoricalFeatures = [ 'make', 'model', 'transmission_type', 'vehicle_style' ]

Question 1: ROC AUC feature importance

ROC AUC could also be used to evaluate feature importance of numerical variables.

Let's do that

    For each numerical variable, use it as score and compute AUC with the above_average variable
    Use the training dataset for that

If your AUC is < 0.5, invert this variable by putting "-" in front

(e.g. -df_train['engine_hp'])

AUC can go below 0.5 if the variable is negatively correlated with the target varialble. You can change the direction of the correlation by negating this variable - then negative correlation becomes positive.

Which numerical variable (among the following 4) has the highest AUC?

    engine_hp
    engine_cylinders
    highway_mpg
    city_mpg
    
ANSWER: engine_hp


In [6]:
#dv = DictVectorizer(sparse=False)

#train_dict = df_train[categoricalFeatures + numericalFeatures].to_dict(orient='records')
#X_train = dv.fit_transform(train_dict)

for feature in numericalFeatures:
    model = LogisticRegression()
    model.fit(df_train[feature].to_frame(), y_train)
    y_train_pred = model.predict(df_train[feature].to_frame())
    #display( y_train )
    #display(y_train_pred )
    fpr, tpr, thresholds = roc_curve(y_train, y_train_pred)
    auc_score = roc_auc_score(y_train, y_train_pred)
    auc_val = metrics.auc(fpr, tpr)
    print( "Feature: ", feature, " auc score: ", auc_score, " auc: ", auc_val )

Feature:  year  auc score:  0.5  auc:  0.5
Feature:  engine_hp  auc score:  0.7944901528558269  auc:  0.7944901528558269
Feature:  engine_cylinders  auc score:  0.6821317811693173  auc:  0.6821317811693173
Feature:  highway_mpg  auc score:  0.5  auc:  0.5
Feature:  city_mpg  auc score:  0.5012800819252432  auc:  0.5012800819252432


In [7]:
print( "Just an experiment")
##for feature in numericalFeatures:
    #y_train = df_train[ 'above_average' ]
    #y_val = df_val[ 'above_average' ]

    #del df_train[ 'above_average' ]
    #del df_val[ 'above_average' ]
    
    ##dv, model = train(df_train[feature].to_frame(), y_train, C=1.0)
    #y_train_pred = predict(df_train[feature].to_frame(), dv, model)
    
    #model = LogisticRegression()
    #model.fit(df_train[feature].to_frame(), y_train)
    #y_train_pred = model.predict(df_train[feature].to_frame())
    #display( y_train )
    #display(y_train_pred )
    ##fpr, tpr, thresholds = roc_curve(y_train, y_train_pred)
    ##auc_score = roc_auc_score(y_train, y_train_pred)
    ##auc_val = metrics.auc(fpr, tpr)
    ##print( "Feature: ", feature, " auc score: ", auc_score, " auc: ", auc_val )
    ##display( df_train[feature].to_frame() )

Just an experiment


Question 2: Training the model

Apply one-hot-encoding using DictVectorizer and train the logistic regression with these parameters:

LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)

What's the AUC of this model on the validation dataset? (round to 3 digits)

    0.678
    0.779
    0.878
    0.979
    
ANSWER: 0.971

In [8]:
train_dict = df_train[categoricalFeatures + numericalFeatures].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

X_train = dv.transform(train_dict)
print( "Got X_train")

Got X_train


In [9]:
val_dict = df_val[categoricalFeatures + numericalFeatures].to_dict(orient='records')

#dv = DictVectorizer(sparse=False)
#dv.fit(val_dict)

X_val = dv.transform(val_dict)
print( "Got X_val")

Got X_val


In [10]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)
y_val_pred = model.predict(X_val)
auc_score = roc_auc_score(y_val, y_val_pred)
print( "auc_score: ", round( auc_score, 3 ) )

auc_score:  0.915


In [11]:
# Variant using Alexey's functions
dv, model = train(df_train, y_train, C=0.1)
y_val_pred = predict(df_val, dv, model)

auc_score = roc_auc_score(y_val, y_val_pred)
print( "auc_score: ", round( auc_score, 3 ) )

auc_score:  0.971


Question 3: Precision and Recall

Now let's compute precision and recall for our model.

    Evaluate the model on all thresholds from 0.0 to 1.0 with step 0.01
    For each threshold, compute precision and recall
    Plot them

At which threshold precision and recall curves intersect?

    0.28
    0.48
    0.68
    0.88
 
ANSWER:0.68

In [12]:
possible_answers = [0.28, 0.48, 0.68, 0.88]
for crt_c in np.arange(0.01,1.0,0.01):
    # avoid the rounding error
    crt_c = round(crt_c,2)
    model = LogisticRegression(solver='liblinear', C=crt_c, max_iter=1000)
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    #fpr, tpr, thresholds = roc_curve(y_val, y_val_pred)
    round_val = 3
    precision = round( precision_score(y_val, y_val_pred, average='weighted'), round_val)
    recall = round( recall_score(y_val, y_val_pred, average='weighted'), round_val)
    if precision == recall:
        print("FOUND crt_c=", crt_c ," precision=", precision, " recall=", recall )
        if crt_c in possible_answers:
            print("FOUND IN LIST ", crt_c) 
    #if precision > recall:
    #    print(" P>R crt_c=", crt_c ," precision=", precision, " recall=", recall )
    #else:
    #    print("P<R crt_c=", crt_c," precision=", precision, " recall=", recall )

FOUND crt_c= 0.31  precision= 0.928  recall= 0.928
FOUND crt_c= 0.33  precision= 0.929  recall= 0.929
FOUND crt_c= 0.38  precision= 0.933  recall= 0.933
FOUND crt_c= 0.4  precision= 0.934  recall= 0.934
FOUND crt_c= 0.41  precision= 0.934  recall= 0.934
FOUND crt_c= 0.42  precision= 0.935  recall= 0.935
FOUND crt_c= 0.43  precision= 0.935  recall= 0.935
FOUND crt_c= 0.44  precision= 0.934  recall= 0.934
FOUND crt_c= 0.45  precision= 0.93  recall= 0.93
FOUND crt_c= 0.49  precision= 0.937  recall= 0.937
FOUND crt_c= 0.5  precision= 0.937  recall= 0.937
FOUND crt_c= 0.54  precision= 0.937  recall= 0.937
FOUND crt_c= 0.56  precision= 0.933  recall= 0.933
FOUND crt_c= 0.57  precision= 0.933  recall= 0.933
FOUND crt_c= 0.58  precision= 0.937  recall= 0.937
FOUND crt_c= 0.61  precision= 0.938  recall= 0.938
FOUND crt_c= 0.63  precision= 0.935  recall= 0.935
FOUND crt_c= 0.64  precision= 0.935  recall= 0.935
FOUND crt_c= 0.65  precision= 0.935  recall= 0.935
FOUND crt_c= 0.66  precision= 0.938

In [13]:
# Variant using Alexey's functions

possible_answers = [0.28, 0.48, 0.68, 0.88]
for crt_c in np.arange(0.01,1.0,0.01):
    # avoid the rounding error
    crt_c = round(crt_c,2)
    
    print(crt_c
    dv, model = train(df_train, y_train, C=crt_c)
    y_val_pred = predict(df_val, dv, model)
    
    #print( y_val_pred)
    ###model = LogisticRegression(solver='liblinear', C=crt_c, max_iter=1000)
    ###model.fit(X_train, y_train)
    ###y_val_pred = model.predict(X_val)
    #fpr, tpr, thresholds = roc_curve(y_val, y_val_pred)
    
    precision = precision_score(y_val, y_val_pred >= 0.5, average='weighted')
    recall = recall_score(y_val, y_val_pred >= 0.5, average='weighted')
    
    #round_val = 16
    #precision = round( precision, round_val )
    #recall = round( precision, round_val )
    if precision == recall:
        print("FOUND crt_c=", crt_c ," precision=", precision, " recall=", recall )
        if crt_c in possible_answers:
            print("FOUND IN LIST ", crt_c) 
    #print( accuracy_score(y_val, y_val_pred >= 0.5) )
#print("Done this step")

Question 4: F1 score

Precision and recall are conflicting - when one grows, the other goes down. That's why they are often combined into the F1 score - a metrics that takes into account both

This is the formula for computing F1:
        
        F1 = 2 * P * R/ (P + R)

Where P is precision and R is recall.

Let's compute F1 for all thresholds from 0.0 to 1.0 with increment 0.01

At which threshold F1 is maximal?

    0.12
    0.32
    0.52
    0.72

ANSWER: 0.72

In [14]:
possible_answers = [0.12, 0.32, 0.52, 0.72]
maxF1 = 0
cForMaxF = 0
for crt_c in np.arange(0.01,1.0,0.01):
    # avoid the rounding error
    crt_c = round(crt_c,2) + 0.0
    model = LogisticRegression(solver='liblinear', C=crt_c, max_iter=1000)
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    #fpr, tpr, thresholds = roc_curve(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, average='weighted')
    recall = recall_score(y_val, y_val_pred, average='weighted')
    round_val = 6
    #precision = round( precision, round_val)
    #recall = round( recall, round_val)
    F1 = 2 * precision * recall / (precision + recall)
    if F1 >= maxF1:
        #print( "Found new max F1; maxF1=", F1, " cForMaxF=",crt_c )
        #if crt_c in possible_answers:
        #    print("FOUND IN LIST ", crt_c)
        if F1 > maxF1:
            maxF1 = F1
            cForMaxF = crt_c
    if crt_c in possible_answers:
        print( "crt_c=", crt_c, " F1=", F1 )
print( "maxF1=", maxF1, " cForMaxF=",cForMaxF )

crt_c= 0.12  F1= 0.9185335193169534
crt_c= 0.32  F1= 0.9283927894250757
crt_c= 0.52  F1= 0.9305310271935054
crt_c= 0.72  F1= 0.9372737196769443
maxF1= 0.9385464589421831  cForMaxF= 0.6


In [15]:
# Variant using Alexey's functions

possible_answers = [0.12, 0.32, 0.52, 0.72]
maxF1 = 0
cForMaxF = 0
for crt_c in np.arange(0.01,1.0,0.01):
    # avoid the rounding error
    crt_c = round(crt_c,2) + 0.0
    
    dv, model = train(df_train, y_train, C=crt_c)
    y_val_pred = predict(df_val, dv, model)
    
    precision = precision_score(y_val, y_val_pred >= 0.5 , average='weighted')
    recall = recall_score(y_val, y_val_pred >= 0.5, average='weighted')
    round_val = 6
    #precision = round( precision, round_val)
    #recall = round( recall, round_val)
    F1 = 2 * precision * recall / (precision + recall)
    if F1 >= maxF1:
        #print( "Found new max F1; maxF1=", F1, " cForMaxF=",crt_c )
        #if crt_c in possible_answers:
        #    print("FOUND IN LIST ", crt_c)
        if F1 > maxF1:
            maxF1 = F1
            cForMaxF = crt_c
    if crt_c in possible_answers:
        print( "crt_c=", crt_c, " F1=", F1 )
print( "maxF1=", maxF1, " cForMaxF=",cForMaxF )
    

crt_c= 0.12  F1= 0.9185335193169534
crt_c= 0.32  F1= 0.9283927894250757
crt_c= 0.52  F1= 0.9305310271935054
crt_c= 0.72  F1= 0.9372737196769443
maxF1= 0.9385464589421831  cForMaxF= 0.6


Question 5: 5-Fold CV

Use the KFold class from Scikit-Learn to evaluate our model on 5 different folds:

KFold(n_splits=5, shuffle=True, random_state=1)

    Iterate over different folds of df_full_train
    Split the data into train and validation
    Train the model on train with these parameters: LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
    Use AUC to evaluate the model on validation

How large is standard devidation of the scores across different folds?

    0.003
    0.030
    0.090
    0.140
    
ANSWER: 0.002

In [16]:
n_splits = 5

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []
C=1.0

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train[ 'above_average' ]
    y_val = df_val[ 'above_average' ]

    del df_train[ 'above_average' ]
    del df_val[ 'above_average' ]

    
    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('%.3f +- %.3f' % (np.mean(scores), np.std(scores)))

0.979 +- 0.002


Question 6: Hyperparemeter Tuning

Now let's use 5-Fold cross-validation to find the best parameter C

    Iterate over the following C values: [0.01, 0.1, 0.5, 10]
    Initialize KFold with the same parameters as previously
    Use these parametes for the model: LogisticRegression(solver='liblinear', C=C, max_iter=1000)
    Compute the mean score as well as the std (round the mean and std to 3 decimal digits)

Which C leads to the best mean score?

    0.01
    0.1
    0.5
    10

If you have ties, select the score with the lowest std. If you still have ties, select the smallest C.

ANSWER: C = 0.1

In [17]:
cList = [0.01, 0.1, 0.5, 10]

for C in cList:
    scores = []
    
    for train_idx, val_idx in kfold.split(df_full_train):
        df_train = df_full_train.iloc[train_idx]
        df_val = df_full_train.iloc[val_idx]

        y_train = df_train[ 'above_average' ]
        y_val = df_val[ 'above_average' ]

        del df_train[ 'above_average' ]
        del df_val[ 'above_average' ]

    
        dv, model = train(df_train, y_train, C=C)
        y_pred = predict(df_val, dv, model)

        auc = roc_auc_score(y_val, y_pred)
        scores.append(auc)

    print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=0.01 0.952 +- 0.003
C=0.1 0.972 +- 0.002
C=0.5 0.977 +- 0.002
C=10 0.981 +- 0.003
